### 0 使用CPU进行模型训练

In [1]:
import keras
from keras.datasets import mnist
import numpy as np
import tensorflow as tf


### 1 数据预处理

In [2]:

(train_images, train_labels), (test_images, test_labels) =mnist.load_data("mnist.npz")
print('训练集',train_images.shape,train_labels.shape)
print('测试集',test_images.shape,test_labels.shape)

img_row,img_col,channel = 28,28,1 # 图像的 高(行row),宽(列col),通道 = 28,28,1

#将数据维度进行处理
train_images = train_images.reshape(train_images.shape[0],img_row,img_col,channel) # 1,28,28,1
test_images = test_images.reshape(test_images.shape[0],img_row,img_col,channel) # 1,28,28,1

train_images = train_images.astype("float32") 
test_images = test_images.astype("float32")

## 进行归一化处理
train_images  /= 255    
test_images /= 255  # test_images = test_images/255

训练集 (60000, 28, 28) (60000,)
测试集 (10000, 28, 28) (10000,)


### 2 构建LeNet-5模型并训练

In [3]:
#构建模型
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28,28,1)),
  keras.layers.Reshape(target_shape=(28,28,1)),
  keras.layers.Conv2D(filters=8, kernel_size=(3,3),padding="same",activation="relu",name='conv1'),
  keras.layers.MaxPooling2D(pool_size=(2,2),name='pool1'),
  keras.layers.Conv2D(filters=16, kernel_size=(3,3),padding="same",activation="relu",name='conv2'),
  keras.layers.MaxPooling2D(pool_size=(2,2),name='pool2'),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu',name='fc1'),
  keras.layers.Dense(10, activation='softmax',name='fc2')
])

# 编译并训练
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels,epochs=5, batch_size=20, validation_data=(test_images,test_labels))



Epoch 1/5
3000/3000 [==============================] - 15s 5ms/step - loss: 0.1531 - accuracy: 0.9532 - val_loss: 0.0574 - val_accuracy: 0.9805
Epoch 2/5
3000/3000 [==============================] - 14s 5ms/step - loss: 0.0539 - accuracy: 0.9832 - val_loss: 0.0380 - val_accuracy: 0.9874
Epoch 3/5
3000/3000 [==============================] - 14s 5ms/step - loss: 0.0364 - accuracy: 0.9884 - val_loss: 0.0393 - val_accuracy: 0.9859
Epoch 4/5
3000/3000 [==============================] - 14s 5ms/step - loss: 0.0268 - accuracy: 0.9916 - val_loss: 0.0380 - val_accuracy: 0.9877
Epoch 5/5
3000/3000 [==============================] - 14s 5ms/step - loss: 0.0216 - accuracy: 0.9931 - val_loss: 0.0377 - val_accuracy: 0.9888


### 3 保存LeNet-5模型为h5格式

In [10]:
# 显示模型信息
model.summary()
# 保存模型
model.save('lenet.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv1 (Conv2D)              (None, 28, 28, 8)         80        
                                                                 
 pool1 (MaxPooling2D)        (None, 14, 14, 8)         0         
                                                                 
 conv2 (Conv2D)              (None, 14, 14, 16)        1168      
                                                                 
 pool2 (MaxPooling2D)        (None, 7, 7, 16)          0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 fc1 (Dense)                 (None, 128)               1

### 4 将h5格式的模型转换为tflite格式

In [13]:

keras_model = tf.keras.models.load_model('lenet.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: D:\USER_T~1\tmpl4t4i2uh\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmpl4t4i2uh\assets


In [14]:
tflite_name = "lenet.tflite"
open(tflite_name, "wb").write(tflite_model)

416048

In [6]:
# TFLite是一个用于在移动设备和边缘设备上部署机器学习模型的框架。在TFLite中，将模型的权重和偏差量化为8位整数可以显著减少模型的大小，从而提高模型的性能和效率。
# 然而，当将模型量化为int8时，偏差的值可能会超过127，这是因为偏差是在训练过程中学习的，并且可能具有比权重更广泛的分布。
# 如果将偏差强制限制为小于等于127，可能会导致精度损失或性能下降。

### 5 将h5模型量化为int8的tflite模型

量化策略：DEFAULT

In [15]:
# 加载Keras模型
keras_model = tf.keras.models.load_model('lenet.h5')
converter_quant = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter_quant.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
  ]
# set the optimization parameters for TensorFlow Lite conversion
converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]

# convert the model to TensorFlow Lite format with float32 activations and int8 weights
quanitfied_defult_model = converter_quant.convert()


INFO:tensorflow:Assets written to: D:\USER_T~1\tmpbc60l9jh\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmpbc60l9jh\assets


In [16]:
#保存转换后的模型
quanitfied_defult_name = "lenet_quanitfied.tflite"
open(quanitfied_defult_name, "wb").write(quanitfied_defult_model)

108040

### 6 模型验证

In [17]:
def evaluate_model(interpreter_path):
    #加载模型并分配张量
    interpreter = tf.lite.Interpreter(model_path=interpreter_path)
    interpreter.allocate_tensors()

    #获得输入输出张量.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    index = input_details[0]['index']
    shape = input_details[0]['shape']
    acc_count = 0
    image_count = test_images.shape[0]
    for i in range(image_count):
        #interpreter.set_tensor(index, test_images[i].reshape(shape).astype("float32"))
        interpreter.set_tensor(index, (test_images[i].reshape(shape)*255).astype("float32"))
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        label = np.argmax(output_data)
        if label == test_labels[i]:
            acc_count += 1
    print("test_images accuracy is {:.2%}".format(acc_count/(image_count)))


In [19]:
print('tflite')
evaluate_model('lenet.tflite')

tflite
test_images accuracy is 98.54%


In [20]:

print('tflite_quanitfied')
evaluate_model('lenet_quanitfied.tflite')

tflite_quanitfied
test_images accuracy is 98.58%
